# Histogram 诊断速查 & 对策

> 目的：通过 **看 hist（直方图）** 快速识别常见数据问题，并给出 **可执行的处理手段**（transform / clip / two-stage / robust loss / scaling / encoding）。  
> 用法：把 `df / target_col / time_col` 改掉，按需运行各节。

---
1. **我在 hist 里看到了什么模式**（问题）  
2. **这会导致什么风险**（对模型/评估/训练）  
3. **我采用什么手段**（具体方法 + 为什么）  


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import PowerTransformer, QuantileTransformer, RobustScaler, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

# ====== TODO: 按你的数据改这里 ======
time_col = None          # e.g. "timestamp"；如果没有时间列，设为 None
target_col = "y"         # e.g. "target"
# df = ...               # 你的 DataFrame
# ====================================

np.random.seed(0)

## 0) 小工具：画 hist + 自动提示

- 支持：原始值 / log1p / winsorize 后对比
- 适合：y、单个 feature、残差（baseline 后）


In [ ]:
def winsorize_series(s: pd.Series, p=0.01):
    """clip 到 [p, 1-p] 分位数，避免极端值主导。"""
    s = s.astype(float)
    lo = s.quantile(p)
    hi = s.quantile(1-p)
    return s.clip(lo, hi)

def plot_hist(s: pd.Series, title: str, bins=50, dropna=True):
    v = s.astype(float)
    if dropna:
        v = v.dropna()
    plt.figure(figsize=(8,4))
    plt.hist(v.values, bins=bins)
    plt.title(title)
    plt.xlabel("value")
    plt.ylabel("count")
    plt.show()

def describe_quick(s: pd.Series):
    v = s.astype(float).dropna()
    return {
        "n": int(v.shape[0]),
        "mean": float(v.mean()),
        "std": float(v.std()),
        "min": float(v.min()),
        "p1": float(v.quantile(0.01)),
        "p50": float(v.quantile(0.50)),
        "p99": float(v.quantile(0.99)),
        "max": float(v.max()),
        "zeros_frac": float((v==0).mean()),
        "neg_frac": float((v<0).mean()),
        "unique": int(v.nunique()),
    }

## 1) 先画 Target（y）的 hist（强制项）

### 常见 hist 模式 → 具体问题 → 处理手段（速查表）

| hist 现象 | 具体问题 | 风险 | 常见对策（可执行） |
|---|---|---|---|
| **heavy tail（长尾）** | 极端值多 | MSE 被少数点主导；线性模型不稳 | winsorize/clip；Huber/Quantile loss；log/yeo-johnson |
| **强右偏（right-skew）** | 分布偏斜 | 线性残差非正态；对均方误差不友好 | log1p（非负）/Yeo-Johnson（可含负）；QuantileTransformer |
| **大量 0（zero-inflated）** | 两种生成机制：0 vs 正值 | 单模型难拟合 | two-stage/hurdle：先分类 P(y>0)，再回归 y|y>0 |
| **双峰/多峰** | regime / mixture | 全局模型不适配 | 分段建模；加 regime 特征；rolling 标准化；cluster 仅用于验证 |
| **离散尖刺（spikes）** | rounding / bucket | 连续假设失效 | 当作类别/ordinal；target encoding；树模型更稳 |
| **明显截断（hard bounds）** | cap/floor | 预测超界/偏差 | 目标变换；使用 bounded model 或 post-clip |


In [ ]:
# --- 运行前检查 ---
assert 'df' in globals(), "请先把 df 读进来并设置 target_col/time_col"

# 如果有时间列：排序
if time_col is not None:
    df = df.sort_values(time_col).reset_index(drop=True)

y = df[target_col].astype(float)

print("y quick stats:", describe_quick(y))
plot_hist(y, "Target y: raw", bins=60)

### 1.1 y 的常用变换对比：raw vs winsorize vs log1p（若适用）

In [ ]:
y_w = winsorize_series(y, p=0.01)
plot_hist(y_w, "Target y: winsorized (p=0.01)", bins=60)

# log1p 只对非负数据适用；若含负，推荐 Yeo-Johnson（见下节）
if (y.dropna() >= 0).all():
    plot_hist(np.log1p(y.dropna()), "Target y: log1p", bins=60)
else:
    print("y 含负值：log1p 不适用。可考虑 Yeo-Johnson（PowerTransformer）。")

### 1.2 如果 y 含负但又偏斜：Yeo-Johnson（不要求非负）

In [ ]:
if y.dropna().shape[0] > 10:
    pt = PowerTransformer(method="yeo-johnson", standardize=False)
    y_yj = pd.Series(pt.fit_transform(y.dropna().to_numpy().reshape(-1,1)).ravel(), index=y.dropna().index)
    plot_hist(y_yj, "Target y: Yeo-Johnson transformed", bins=60)

## 2) 通过 hist 发现的问题：怎么“落地到处理手段”

下面给你几段 **可直接复制到面试 notebook** 的处理模块。


### 2.1 长尾 / 极端值：Winsorize / Clip + Robust Loss（示例：Ridge + clip）

> 注：真实面试里你不一定要上复杂 loss；最常用、最快的是 **clip** 或 **winsorize**。


In [ ]:
# 示例：对 y 做 winsorize，然后做一个简单 Ridge baseline（仅演示流程）
# 你也可以把 winsorize 用在某些 feature 上。

split_idx = int(0.8 * len(df))
y_train = y.iloc[:split_idx]
y_test  = y.iloc[split_idx:]

y_train_w = winsorize_series(y_train, p=0.01)
y_test_w  = y_test.clip(y_train_w.min(), y_train_w.max())  # 用 train 的界限 clip test（防泄露）

print("Train y range after winsorize:", float(y_train_w.min()), float(y_train_w.max()))
plot_hist(y_train_w, "Train y (winsorized)", bins=60)

### 2.2 偏斜分布：log / Yeo-Johnson / QuantileTransformer

- **log1p**：适用于 `x >= 0`
- **Yeo-Johnson**：允许负值
- **QuantileTransformer**：把分布“拉直”到接近正态/均匀（可能抹掉 level 信息，时序要谨慎）


In [ ]:
# 针对一个 feature 的示例（你可以换成任意 feature）
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols = [c for c in numeric_cols if c != target_col]

if len(numeric_cols) == 0:
    print("没有数值特征可演示。")
else:
    feat = numeric_cols[0]
    x = df[feat].astype(float)
    print("feature:", feat, "stats:", describe_quick(x))
    plot_hist(x, f"Feature {feat}: raw", bins=60)

    x_w = winsorize_series(x, p=0.01)
    plot_hist(x_w, f"Feature {feat}: winsorized", bins=60)

    if (x.dropna() >= 0).all():
        plot_hist(np.log1p(x.dropna()), f"Feature {feat}: log1p", bins=60)
    else:
        pt = PowerTransformer(method="yeo-johnson", standardize=False)
        x_yj = pd.Series(pt.fit_transform(x.dropna().to_numpy().reshape(-1,1)).ravel(), index=x.dropna().index)
        plot_hist(x_yj, f"Feature {feat}: Yeo-Johnson", bins=60)

    qt = QuantileTransformer(output_distribution="normal", random_state=0)
    x_qt = pd.Series(qt.fit_transform(x.dropna().to_numpy().reshape(-1,1)).ravel(), index=x.dropna().index)
    plot_hist(x_qt, f"Feature {feat}: Quantile->Normal", bins=60)

### 2.3 大量 0（zero-inflated）：two-stage / hurdle（模板）

如果 hist 显示 y 有“尖锐的大量 0”，通常暗示 **两种机制**：  
- 先决定是否为 0（分类）  
- 再决定正值的大小（回归）

下面只给你结构模板（面试里够用）。


In [ ]:
# 判断 y 是否明显 zero-inflated（一个启发式）
zero_frac = float((y==0).mean())
print("zero fraction:", zero_frac)

if zero_frac > 0.2:
    print("提示：y 可能 zero-inflated，考虑 two-stage/hurdle。")
else:
    print("提示：y 的 0 比例不高，两阶段未必必要。")

## 3) “看很多 hist”是否必要？（批量扫描的正确方式）

面试里不建议对每个特征都画图，但可以做一个**快速统计扫描**：
- 0 比例
- 负值比例
- unique（离散程度）
- p1/p99 距离（长尾程度）

然后只挑 Top-K “最异常”的特征画 hist。


In [ ]:
# 批量扫描数值特征（只输出一个表）
rows = []
for c in numeric_cols[:200]:  # 防止列太多拖慢，可调
    s = df[c]
    d = describe_quick(s)
    d["col"] = c
    # 一个简单的 tail 指标：p99 - p50 相对尺度
    d["tail_score"] = (d["p99"] - d["p50"]) / (abs(d["p50"]) + 1e-9)
    rows.append(d)

scan = pd.DataFrame(rows).sort_values("tail_score", ascending=False)
scan.head(15)

### 3.1 只画 Top-K “最异常”的特征 hist（推荐）

In [ ]:
top_k = 6
for c in scan.head(top_k)["col"].tolist():
    plot_hist(df[c], f"Top tail feature: {c}", bins=60)

## 4) 时序数据特别注意：train/test 分布漂移（同一特征在不同时段分布不同）

你可以对比：
- Train vs Test 的 hist（或统计量）
- 若差异显著 → 可能存在 regime shift，normalize/transform 需要更谨慎


In [ ]:
if len(numeric_cols) > 0:
    feat = numeric_cols[0]
    split_idx = int(0.8 * len(df))
    x_train = df[feat].iloc[:split_idx].astype(float)
    x_test  = df[feat].iloc[split_idx:].astype(float)

    print("train stats:", describe_quick(x_train))
    print("test  stats:", describe_quick(x_test))

    plt.figure(figsize=(10,4))
    plt.hist(x_train.dropna().values, bins=60, alpha=0.6, label="train")
    plt.hist(x_test.dropna().values, bins=60, alpha=0.6, label="test")
    plt.title(f"Train vs Test Distribution: {feat}")
    plt.xlabel("value")
    plt.ylabel("count")
    plt.legend()
    plt.show()

## 5) 把“分布处理”放进 sklearn Pipeline（防泄露的标准做法）

关键点：
- Imputer / Scaler / Transformer 只在 **train fit**
- test 只做 transform
- 时序 split 不 random

下面是一个通用模板：数值列做 `impute -> robust scale`，并可插入 power transform / quantile transform。


In [ ]:
# 一个简化模板：Ridge + 数值特征处理（你可替换成别的模型）
numeric_cols = [c for c in df.select_dtypes(include=[np.number]).columns if c != target_col]
X = df[numeric_cols].copy()
y = df[target_col].astype(float)

split_idx = int(0.8 * len(df))
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

numeric_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", RobustScaler()),  # 对长尾更稳
    # ("power", PowerTransformer(method="yeo-johnson", standardize=True)),  # 可选
    # ("qt", QuantileTransformer(output_distribution="normal", random_state=0)),  # 可选：时序谨慎
])

preprocess = ColumnTransformer(
    transformers=[("num", numeric_pipe, numeric_cols)],
    remainder="drop"
)

model = Ridge(alpha=1.0)

pipe = Pipeline(steps=[("preprocess", preprocess),
                      ("model", model)])

pipe.fit(X_train, y_train)
pred = pipe.predict(X_test)

rmse = mean_squared_error(y_test, pred, squared=False)
print("Ridge baseline RMSE:", rmse)

## 6)
- 我先用 y 的直方图判断是否存在长尾/偏斜/多峰/大量0等结构  
- 对长尾：优先考虑 winsorize/clip 或 robust 方法，避免少数极端点主导  
- 对偏斜：根据是否含负选择 log1p 或 Yeo-Johnson；必要时用 quantile transform  
- 对大量 0：考虑 two-stage/hurdle（先分类再回归）  
- 对多峰/分段：优先做 time-aware 的验证与特征（rolling / regime indicator），clustering 仅用于验证而非默认步骤  
- 所有 transform 都放进 sklearn Pipeline，并且只在 train 上 fit，避免泄露
